## Question 1

In [46]:
!pip install mlflow jupyter scikit-learn pandas seaborn hyperopt xgboost fastparquet boto3 optuna

     |████████████████████████████████| 390 kB 2.6 MB/s            
You should consider upgrading via the '/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/bin/python -m pip install --upgrade pip' command.


In [4]:
import mlflow
print(mlflow.__version__)

2.3.2


Answer: 2.3.2

## Question 2

In [16]:
import os
!mkdir os.getcwd()+'/data'

In [38]:
# Download data
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
!wget -P data https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-03.parquet

--2023-06-04 01:27:17--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.227.44.79, 13.227.44.50, 13.227.44.193, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.227.44.79|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1254291 (1.2M) [binary/octet-stream]
Saving to: 'data/green_tripdata_2022-01.parquet.1'

green_tripdata_2022 100%[===================>]   1.20M  --.-KB/s    in 0.1s    

2023-06-04 01:27:17 (8.52 MB/s) - 'data/green_tripdata_2022-01.parquet.1' saved [1254291/1254291]

--2023-06-04 01:27:17--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2022-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 13.227.44.79, 13.227.44.50, 13.227.44.193, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|13.227.44.79|:443... connected.
HTTP req

In [40]:
# Call preprocess script
!python preprocess_data.py --raw_data_path data --dest_path ./output


In [41]:
os.stat('output/dv.pkl').st_size

153660

Answer: 153660 bytes == 154 mega-bytes

## Question 3

In [50]:
!python train.py --data_path ./output

2023/06/04 01:54:32 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-packages/_distutils_hack/__init__.py:36: UserWarning: Setuptools is replacing distutils."


In [51]:
!mlflow ui


[2023-06-04 01:55:12 -0500] [31689] [INFO] Starting gunicorn 20.1.0
[2023-06-04 01:55:12 -0500] [31689] [INFO] Listening at: http://127.0.0.1:5000 (31689)
[2023-06-04 01:55:12 -0500] [31689] [INFO] Using worker: sync
[2023-06-04 01:55:12 -0500] [31690] [INFO] Booting worker with pid: 31690
[2023-06-04 01:55:12 -0500] [31691] [INFO] Booting worker with pid: 31691
[2023-06-04 01:55:12 -0500] [31692] [INFO] Booting worker with pid: 31692
[2023-06-04 01:55:12 -0500] [31693] [INFO] Booting worker with pid: 31693
[2023-06-04 02:32:03 -0500] [31689] [CRITICAL] WORKER TIMEOUT (pid:31690)
[2023-06-04 02:32:03 -0500] [31689] [CRITICAL] WORKER TIMEOUT (pid:31691)
[2023-06-04 02:32:03 -0500] [31689] [CRITICAL] WORKER TIMEOUT (pid:31692)
[2023-06-04 02:32:03 -0500] [31689] [CRITICAL] WORKER TIMEOUT (pid:31693)
[2023-06-04 02:32:03 -0500] [31693] [INFO] Worker exiting (pid: 31693)
[2023-06-04 02:32:03 -0500] [31691] [INFO] Worker exiting (pid: 31691)
[2023-06-04 02:32:03 -0500] [31690] [INFO] Worker

In [52]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Retrieve the experiment ID from its name.
experiment_name = "random-forest"
experiment = client.get_experiment_by_name(experiment_name)
experiment_id = experiment.experiment_id

runs = client.search_runs(experiment_ids=[experiment_id])
for run in runs:
    params = client.get_run(run.info.run_id).data.params
    max_depth = params.get("max_depth")
    print(f"max_depth for run is {run_id}: {max_depth}")

max_depth for run 9acd543b186544eab564a7d0cedf7b8d: 10
max_depth for run 567e1e8b071142a7ad249072d22ec096: 10


Answer: 10

## Question 4

In [54]:
!python hpo.py --data_path ./output

Traceback (most recent call last):
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/homework/homework_2/hpo.py", line 12, in <module>
    mlflow.set_experiment("random-forest-hyperopt")
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-packages/mlflow/tracking/fluent.py", line 114, in set_experiment
    experiment = client.get_experiment_by_name(experiment_name)
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-packages/mlflow/tracking/client.py", line 461, in get_experiment_by_name
    return self._tracking_client.get_experiment_by_name(name)
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-packages/mlflow/tracking/_tracking_service/client.py", line 220, in get_experiment_by_name
    return self.store.get_experiment_by_name(name)
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-p

Answer: 2.45

## Question 5

In [55]:
!python register_model.py --data_path ./output

Traceback (most recent call last):
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/homework/homework_2/register_model.py", line 16, in <module>
    mlflow.set_experiment(EXPERIMENT_NAME)
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-packages/mlflow/tracking/fluent.py", line 114, in set_experiment
    experiment = client.get_experiment_by_name(experiment_name)
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-packages/mlflow/tracking/client.py", line 461, in get_experiment_by_name
    return self._tracking_client.get_experiment_by_name(name)
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site-packages/mlflow/tracking/_tracking_service/client.py", line 220, in get_experiment_by_name
    return self.store.get_experiment_by_name(name)
  File "/Users/evanhofmeister/PycharmProjects/dezoomcamp-mlops-homework/venv/lib/python3.9/site

Answer: 2.29

## Question 6

Answer: All of the above